In [505]:
import numpy as np

In [506]:
import random

class Cell:
    """Cell class that defines each walkable Cell on the grid"""
    def __init__(self, x: int, y: int):
        self.x = x
        self.y = y
        self.visited = False
        self.walls = [True, True, True, True] # Left, Right, Up, Down


    def getChildren(self, grid: list) -> list:
        """Check if the Cell has any surrounding unvisited Cells that are walkable"""
        a = [(1, 0), (-1,0), (0, 1), (0, -1)]
        children = []
        for x, y in a:
            if self.x+x in [len(grid), -1] or self.y+y in [-1, len(grid)]:
                continue
            
            child = grid[self.y+y][self.x+x]
            if child.visited:
                continue
            children.append(child)
        return children


def removeWalls(current: Cell, choice: Cell):
    """Removeing the wall between two Cells"""
    if choice.x > current.x:     
        current.walls[1] = False
        choice.walls[0] = False
    elif choice.x < current.x:
        current.walls[0] = False
        choice.walls[1] = False
    elif choice.y > current.y:
        current.walls[3] = False
        choice.walls[2] = False
    elif choice.y < current.y:
        current.walls[2] = False
        choice.walls[3] = False


def drawWalls(grid: list, binGrid: list) -> list:
    """Draw existing walls around Cells"""
    for yindex, y in enumerate(grid):
        for xindex, x in enumerate(y):
            for i, w in enumerate(x.walls):
                if i == 0 and w:
                    binGrid[yindex*2+1][xindex*2] = '%'
                if i == 1 and w:
                    binGrid[yindex*2+1][xindex*2+2] = '%'
                if i == 2 and w:
                    binGrid[yindex*2][xindex*2+1] = '%'
                if i == 3 and w:
                    binGrid[yindex*2+2][xindex*2+1] = '%'
    return binGrid


def drawBorder(grid: list) -> list:
    """Draw a border around the maze"""
    length = len(grid)
    for row in grid:
        row[0] = row[length-1] = '%'
        
    grid[0] = grid[length-1] = ['%'] * length
    return grid


def displayMaze(grid: list):
    """Draw the maze using ASCII characters and display the maze"""
    binGrid = []
    length = len(grid)*2+1
    for x in range(length):
        if x % 2 == 0:
            binGrid.append([' ' if x % 2 != 0 else '%' for x in range(length)])
        else:
            binGrid.append([' '] * length)
    
    binGrid = drawWalls(grid, binGrid)
            
    binGrid = drawBorder(binGrid)

    print('\n'.join([''.join(x) for x in binGrid]))
    return binGrid


# Request the user to input a maze size and initialise the maze, stack and initial Cell
# size = int(input('Enter a maze size: '))

size = 5
grid = [[Cell(x, y) for x in range(size)] for y in range(size)]
current = grid[0][0]
stack = []


# Main loop to generate the maze
while True:
    current.visited = True
    children = current.getChildren(grid)

    if children:
        choice = random.choice(children)
        choice.visited = True

        stack.append(current)

        removeWalls(current, choice)

        current = choice
    
    elif stack:
        current = stack.pop()
    else:
        break

grid_display = displayMaze(grid)

%%%%%%%%%%%
% %       %
% %%%%% % %
%   %   % %
%%% % %%% %
% %   % % %
% %%%%% % %
% %     % %
% % % %%% %
%   %     %
%%%%%%%%%%%


## Logic to remove random parts of the walls. 
### If you wanna remove even more % from the grid, just add a constant to rnd_count (fro example: rnd_count = rnd_count + 2)

In [507]:
new_grid = [grid_display[0]]

# first row and last row are ignored
for curr_row in grid_display[1:len(grid_display) - 1]:
    count = curr_row.count("%") - 2
    rnd_count = round(np.random.uniform(low = 0, high = count/1.5))
    
    list_of_walls = []
    for index in range(1, len(curr_row) - 1):
        if curr_row[index] == "%":
            list_of_walls.append(index)
    
    indexes_to_be_replaced = np.random.choice(list_of_walls, rnd_count, replace = False)
#     print(indexes_to_be_replaced)
#     print("Previous: ", curr_row)
    for idx in indexes_to_be_replaced:
        curr_row[idx] = " "
#     print("After   : ", curr_row)
    new_grid.append(curr_row)
new_grid.append(grid_display[-1])

In [508]:
print('\n'.join([''.join(x) for x in new_grid]))

%%%%%%%%%%%
% %       %
%  % %  % %
%   %     %
%%% % %%  %
% %       %
% % %%% % %
% %     % %
% % %  %  %
%   %     %
%%%%%%%%%%%


# Logic to add food pellets. 

### if you wanna add more food, just increase 0.3 to some other higher number.

In [509]:
# addign food pellets:
for curr_row in new_grid:
    for index, element in enumerate(curr_row):
        if element == " ":
            if np.random.random() < 0.3:
                curr_row[index] = "."

In [510]:
print('\n'.join([''.join(x) for x in new_grid]))

%%%%%%%%%%%
%.%  ...  %
%  % %  % %
%  .%     %
%%% %.%%. %
% %.    ..%
% % %%% % %
% %.  . % %
% % %  % .%
%   %  ...%
%%%%%%%%%%%


# Logic to add one pacman in the grid

### the pacman is just initiated in the top-left corner of the grid

In [511]:
new_grid[1][1] = "P"
print('\n'.join([''.join(x) for x in new_grid]))

%%%%%%%%%%%
%P%  ...  %
%  % %  % %
%  .%     %
%%% %.%%. %
% %.    ..%
% % %%% % %
% %.  . % %
% % %  % .%
%   %  ...%
%%%%%%%%%%%


# Logic to add x ghosts in the grid

### the ghosts are just initiated in the bottom-right corners of the grid

In [512]:
# the logic will support for any number of ghosts.
# The ghosts will be initiated in the bottom part of the randomly generated grid.

width = len(new_grid[0]) 
height = len(new_grid)

empty_locations = []
for x in range(width):
    for y in range(height):
        if new_grid[x][y] == " ":
            empty_locations.append((x, y))

In [513]:
Num_Ghosts = 1
PowerPallets = 2
for i in range(Num_Ghosts):
    curr_empty_location = empty_locations[-1 * (i + 1)]
    new_grid[curr_empty_location[0]][curr_empty_location[1]] = "G"
    empty_locations.remove((curr_empty_location[0], curr_empty_location[1]))

for i in range(PowerPallets):
    curr_empty_location = random.choice(empty_locations)
    new_grid[curr_empty_location[0]][curr_empty_location[1]] = "o"
    empty_locations.remove((curr_empty_location[0], curr_empty_location[1]))

In [514]:
print('\n'.join([''.join(x) for x in new_grid]))

%%%%%%%%%%%
%P%  ...  %
%  % %  % %
%  .%     %
%%% %.%%. %
%o%.    ..%
% % %%% % %
% %.  . % %
% % %  % .%
%   %oG...%
%%%%%%%%%%%


In [515]:
# with open("layouts/genSmallG2", )